In [20]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import random
import pandas as pd
import copy 
import numpy as np
import pickle
headOption = webdriver.FirefoxOptions()
headOption.add_argument('-headless')

driver = webdriver.Firefox(executable_path='/Users/william/Downloads/geckodriver', options=headOption)

#driver = webdriver.Firefox(executable_path='/Users/william/Downloads/geckodriver')


In [21]:
class card:
     def __init__(self, name, cost, move_f, move_ch, move_tm, level, a_iv, d_iv, hp_iv, tm_flag, shiny, attack_stg, def_stg, health, energy, bonus_shield, status):
        self.name = name
        self.cost = cost
        self.move_f = move_f
        self.move_ch = move_ch
        self.move_tm = move_tm
        self.level = level
        self.a_iv = a_iv
        self.d_iv = d_iv
        self.hp_iv = hp_iv
        self.tm_flag = tm_flag
        self.attack_stg = attack_stg
        self.def_stg = def_stg
        self.shiny = shiny
        self.health = health
        self.energy = energy
        self.bonus_shield = bonus_shield
        self.status = status    
class card:
     def __init__(self, name, cost, move_f, move_ch, move_tm, level, a_iv, d_iv, hp_iv, tm_flag, shiny, attack_stg, def_stg, health, energy, bonus_shield, status):
        self.name = name
        self.cost = cost
        self.move_f = move_f
        self.move_ch = move_ch
        self.move_tm = move_tm
        self.level = level
        self.a_iv = a_iv
        self.d_iv = d_iv
        self.hp_iv = hp_iv
        self.tm_flag = tm_flag
        self.attack_stg = attack_stg
        self.def_stg = def_stg
        self.shiny = shiny
        self.health = health
        self.energy = energy
        self.bonus_shield = bonus_shield
        self.status = status    
def card_from_string(pvpokestring):
    l = pvpokestring.split(',')
    return(card(
    name = l[0],
    cost = l[8],
     move_f = l[1],
     move_ch = l[2],
     move_tm = l[3],
     level = l[4],
    a_iv = l[5], 
    d_iv = l[6], 
    hp_iv = l[7],
    tm_flag = 0,
    attack_stg = 0,
    def_stg = 0,
    shiny = 0,
    health = 0,
    energy = 0,
    bonus_shield= 0,
    status = 1
        
    )
          )




In [22]:
def oneVone(battler1, battler2):
    
    if battler1.shiny == 1:
        BUFF1 = 5
    else:
        BUFF1 = 4
        
    if battler2.shiny == 1:
        BUFF2 = 5
    else:
        BUFF2 = 4
    SHIELD1 = 1+battler1.bonus_shield
    SHIELD2 = 1+battler2.bonus_shield
    
    if battler1.tm_flag == 0:
        TM1 = 0
    else:
        TM1 = battler1.move_tm

    if battler2.tm_flag == 0:
        TM2 = 0
    else:
        TM2 = battler2.move_tm

        
        
    url = f'http://localhost/pvpoke/src/battle/10000/{battler1.name}-{battler1.level}-{battler1.a_iv}-{battler1.d_iv}-{battler1.hp_iv}-{BUFF1}-{BUFF1}-1-0/{battler2.name}-{battler2.level}-{battler2.a_iv}-{battler2.d_iv}-{battler2.hp_iv}-{BUFF2}-{BUFF2}-1-0/{SHIELD1}{SHIELD2}/{battler1.move_f}-{battler1.move_ch}-{TM1}/{battler2.move_f}-{battler2.move_ch}-{TM2}/{battler1.health}-{battler2.health}/{battler1.energy}-{battler2.energy}/'
    
    driver.get(url)
    
    
    result = driver.find_element_by_xpath("//div[@class='summary section white']").text.split()
    while len(result) == 0:
        result = driver.find_element_by_xpath("//div[@class='summary section white']").text.split()
    
    if result[0] == 'Simultaneous':
        winner = 0
        survivorhp = None
        survivorenergy = None
    else:
        battle_score = int(driver.find_element_by_xpath("/html/body/div/div/div[4]/div[2]/div[5]/div[1]/span[3]").text)
        survivor = driver.find_element_by_xpath("/html/body/div/div/div[4]/div[3]/div[2]/p/span").text.split()
        survivorhp = int(survivor[1][1:])
        survivorenergy = int(survivor[3])
        survivorshields = int(survivor[5])

        if battle_score > 500:
            winner = 1
        else:
            winner = 2 
    
    
    return([winner, survivorhp, survivorenergy,survivorshields])

In [23]:
def battle(team1, team2):
    team1_live = copy.deepcopy(team1)
    team2_live = copy.deepcopy(team2)
    current_team1 = next((x for x in team1_live if x.status == 1), None)
    current_team2 = next((x for x in team2_live if x.status == 1), None)

    while (current_team1 != None ) & (current_team2 != None):
        battle_result = oneVone(current_team1,current_team2)
        if battle_result[0] == 0:
            print("Simultaneous KO by " +current_team1.name +" and " + current_team2.name)
            current_team1.status = 0
            current_team2.status = 0
        elif battle_result[0] == 1:
            print(current_team1.name +" knocks out " + current_team2.name)
            current_team2.status = 0
            current_team1.health = battle_result[1]
            current_team1.energy = battle_result[2]
            current_team1.bonus_shield = battle_result[3]-1

        elif battle_result[0] == 2:
            print(current_team2.name +" knocks out " + current_team1.name)
            current_team1.status = 0
            current_team2.health = battle_result[1]
            current_team2.energy = battle_result[2]
            current_team2.bonus_shield = battle_result[3]-1
            
        current_team1 = next((x for x in team1_live if x.status == 1), None)
        current_team2 = next((x for x in team2_live if x.status == 1), None)

    if (current_team1 == None) & (current_team2 == None):
        print("It's a tie!")
    elif (current_team1 != None):
        print("Team 1 wins!")
    elif (current_team2 != None):
        print("Team 2 wins!")

In [168]:
team1 = [charizard, venusaur, blastoise]

In [169]:
team2 = [arcanine,lapras, exeggutor]

In [172]:
battle(team1, team2)

arcanine knocks out charizard
venusaur knocks out arcanine
venusaur knocks out lapras
exeggutor knocks out venusaur
blastoise knocks out exeggutor
Team 1 wins!


In [25]:
with open('card_list.pkl', 'rb') as f:
    card_list = pickle.load(f)

In [26]:
with open('pve_card_list.pkl', 'rb') as f:
    pve_card_list = pickle.load(f)